Task 1

In [2]:
import numpy as np
seed_value = 2312
n=320
rng = np.random.default_rng(seed_value)

tickets = []
for i in range(1, n+1):
    ticket_id = f"{seed_value}-{i:04d}"

    routes = ["NYC-LAX", "LHR-JFK", "SFO-SEA", "DXB-SIN", "MAD-ROM"]
    route_index=(i + seed_value) % 5
    route = routes[route_index]
    
    days = ["Mon", "Tue", "Wed", "Thu", "Fri", "Sat", "Sun"]
    day_index = (i + seed_value) % 7
    day = days[day_index]
    
    days_to_departure = 1 + ((i * 3 + seed_value) % 60)
    
    classes = ["economy", "premium", "business"]
    class_index = (i * 2 + seed_value) % 3
    ticket_class = classes[clas_index]
    
    base = 120 + (days_to_departure * -1.5)
    
    route_adj_values = [140, 220, 60, 180, 80]
    route_adj = route_adj_values[route_index]
    
    class_adj_values = [0, 80, 220]
    class_adj = class_adj_values[class_index]
    
    noise = rng.normal(0, 25)
    
    price_usd = round(base + route_adj + class_adj + noise, 2)
    
    if i % 28 == 0:
        price_usd = ""
    if i % 45 == 0:
        price_usd = price_usd * -1
    if i % 37 == 0:
        ticket_class = ticket_class.upper()
    
    ticket = {
        "ticket_id": ticket_id,
        "route": route,
        "day": day,
        "days_to_departure": days_to_departure,
        "class": ticket_class,
        "price_usd": price_usd
    }

    tickets.append(ticket)

print("Count of records: ", len(tickets))
print("The first five entries: ")
for t in tickets[:5]:
    print(t)



Count of records:  320
The first five entries: 
{'ticket_id': '2312-0001', 'route': 'DXB-SIN', 'day': 'Thu', 'days_to_departure': 36, 'class': 'premium', 'price_usd': 303.6}
{'ticket_id': '2312-0002', 'route': 'MAD-ROM', 'day': 'Fri', 'days_to_departure': 39, 'class': 'premium', 'price_usd': 125.63}
{'ticket_id': '2312-0003', 'route': 'NYC-LAX', 'day': 'Sat', 'days_to_departure': 42, 'class': 'premium', 'price_usd': 435.39}
{'ticket_id': '2312-0004', 'route': 'LHR-JFK', 'day': 'Sun', 'days_to_departure': 45, 'class': 'premium', 'price_usd': 334.82}
{'ticket_id': '2312-0005', 'route': 'SFO-SEA', 'day': 'Mon', 'days_to_departure': 48, 'class': 'premium', 'price_usd': 95.01}


In [4]:
#task 2
cleaned_tickets = []
for ticket in tickets:
    price = ticket["price_usd"]

    if not isinstance(price, (int, float)):
        continue
    if price < 0:
        continue
    if price == "":
        continue

    ticket["class"] = ticket["class"].lower()
    cleaned_tickets.append(ticket)


print("Valid records:", len(cleaned_tickets))

for ticket in tickets[:5]:
    print(ticket)

Valid records: 302
{'ticket_id': '2312-0001', 'route': 'DXB-SIN', 'day': 'Thu', 'days_to_departure': 36, 'class': 'premium', 'price_usd': 303.6}
{'ticket_id': '2312-0002', 'route': 'MAD-ROM', 'day': 'Fri', 'days_to_departure': 39, 'class': 'premium', 'price_usd': 125.63}
{'ticket_id': '2312-0003', 'route': 'NYC-LAX', 'day': 'Sat', 'days_to_departure': 42, 'class': 'premium', 'price_usd': 435.39}
{'ticket_id': '2312-0004', 'route': 'LHR-JFK', 'day': 'Sun', 'days_to_departure': 45, 'class': 'premium', 'price_usd': 334.82}
{'ticket_id': '2312-0005', 'route': 'SFO-SEA', 'day': 'Mon', 'days_to_departure': 48, 'class': 'premium', 'price_usd': 95.01}


In [10]:
# task 3 

prices = np.array([ticket["price_usd"] for ticket in cleaned_tickets])
days = np.array([ticket["day"] for ticket in cleaned_tickets])
mean_prices = np.mean(prices)
std_prices = np.std(prices)
print(mean_prices)
print(std_prices)

307.22943708609273
115.40513485833237


In [11]:
unique_days = np.unique(days)
unique_days

array(['Fri', 'Mon', 'Sat', 'Sun', 'Thu', 'Tue', 'Wed'], dtype='<U3')

In [24]:
revenue_per_day = np.array([prices[days == d].sum() for d in unique_days])
ticket_per_day = np.array([np.sum(days == d) for d in unique_days])


In [19]:
total_revenue = prices.sum()
daily_total = revenue_per_day.sum()

assert np.isclose(total_revenue, daily_total)

In [21]:
for day, revenue, count in zip(unique_days, revenue_per_day, ticket_per_day):
    print(f"{day}: Total revenue = {revenue :.2f}, Ticket Sold = {count:.2f}")

Fri: Total revenue = 13736.29, Ticket Sold = 45.00
Mon: Total revenue = 13548.82, Ticket Sold = 45.00
Sat: Total revenue = 14367.91, Ticket Sold = 45.00
Sun: Total revenue = 13782.55, Ticket Sold = 45.00
Thu: Total revenue = 13643.49, Ticket Sold = 45.00
Tue: Total revenue = 13316.07, Ticket Sold = 44.00
Wed: Total revenue = 10388.16, Ticket Sold = 33.00


In [22]:
print(f"Total Revenue from prices array: {total_revenue:.2f}")

Total Revenue from prices array: 92783.29


In [29]:
# task4
thresold = np.percentile(prices, 90)
print(thresold)
high_prices = prices[prices >= thresold]
print(len(high_prices))

466.612
31


array([483.23, 517.02, 497.16, 568.01, 530.56, 525.19, 466.68, 475.07,
       500.2 , 572.88, 548.57, 484.96, 495.68, 486.43, 494.68, 490.4 ,
       515.44, 553.29, 485.45, 480.83, 486.48, 509.86, 499.36, 549.86,
       475.19, 500.75, 484.78, 550.62, 504.29, 563.86, 476.76])

In [30]:
print(np.all(high_prices >= thresold))

True


In [31]:
#task 5
report = {
    "total_tickets": np.size(tickets),
    "cleaned_tickets": len(cleaned_tickets),
    "mean_price": mean_prices,
    "std_price": std_prices,
    "daily_totals": daily_total,
    "high_price_count": len(high_prices)
}

In [34]:
assert report["cleaned_tickets"] <= report["total_tickets"], \
       "Validation failed!"

In [35]:
report_str = f"""
 FINAL REPORT
Total tickets: {report['total_tickets']}
Cleaned tickets: {report['cleaned_tickets']}
Mean price: {report['mean_price']:.2f}
Std price: {report['std_price']:.2f}
High price tickets: {report['high_price_count']}

Daily totals:
{report['daily_totals']}
"""

print(report_str)



 FINAL REPORT
Total tickets: 320
Cleaned tickets: 302
Mean price: 307.23
Std price: 115.41
High price tickets: 31

Daily totals:
92783.29000000001

